In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from arguments import Arguments
from cnn import CNN
from distributor import get_distributed_data, get_distributed_data_using_loader
from distributor import get_fog_graph
import os
import syft as sy
import torch
from torchvision import datasets, transforms
from train import fl_train as train, fog_train
from train import test

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [3]:
# Setups
args = Arguments()
USE_CUDA = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
device = torch.device("cuda" if USE_CUDA else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}
kwargs = {}

In [4]:
ckpt_path = '../ckpts'
dataset = 'mnist'
clf_type = 'cnn'
paradigm = 'fl'
model_name = '{}_{}_{}'.format(dataset, clf_type, paradigm)
init_path = os.path.join(ckpt_path, 'mnist_cnn_fl.init')
best_path = os.path.join(ckpt_path, model_name + '.best')
stop_path = os.path.join(ckpt_path, model_name + '.stop')


In [5]:
# Setup hook to support FL
hook = sy.TorchHook(torch)
# Define workers
workers = list()
for id_ in range(args.num_workers):
    workers.append(sy.VirtualWorker(hook, id=str(id_)))

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.num_train, shuffle=False, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.num_test, shuffle=True, **kwargs)

for data, target in train_loader:
    X_train = data
    y_train = target

for data, target in test_loader:
    X_test = data
    y_test = target

print('X_train: {}'.format(X_train.shape))
print('y_train: {}'.format(y_train.shape))

print('X_test: {}'.format(X_test.shape))
print('y_test: {}'.format(y_test.shape))


X_train: torch.Size([60000, 1, 28, 28])
y_train: torch.Size([60000])
X_test: torch.Size([10000, 1, 28, 28])
y_test: torch.Size([10000])


In [6]:
# prepare graph and data
fog_graph, workers = get_fog_graph(hook, args.num_workers, args.num_clusters,
                                   args.shuffle_workers, args.uniform_clusters, fog=False)
X_trains, y_trains = get_distributed_data(X_train, y_train, args.num_workers)

print(fog_graph)

{'L1_W0': ['L0_W0', 'L0_W1', 'L0_W2', 'L0_W3', 'L0_W4', 'L0_W5', 'L0_W6', 'L0_W7', 'L0_W8', 'L0_W9', 'L0_W10', 'L0_W11', 'L0_W12', 'L0_W13', 'L0_W14', 'L0_W15', 'L0_W16', 'L0_W17', 'L0_W18', 'L0_W19', 'L0_W20', 'L0_W21', 'L0_W22', 'L0_W23', 'L0_W24', 'L0_W25', 'L0_W26', 'L0_W27', 'L0_W28', 'L0_W29', 'L0_W30', 'L0_W31', 'L0_W32', 'L0_W33', 'L0_W34', 'L0_W35', 'L0_W36', 'L0_W37', 'L0_W38', 'L0_W39', 'L0_W40', 'L0_W41', 'L0_W42', 'L0_W43', 'L0_W44', 'L0_W45', 'L0_W46', 'L0_W47', 'L0_W48', 'L0_W49']}


In [9]:
best = 0
# Fire the engines
model = CNN().to(device)
if args.load_init:
    model.load_state_dict(torch.load(init_path))
    print('Load init: {}'.format(init_path))
elif args.save_init:
    torch.save(model.state_dict(), init_path)
    print('Save init: {}'.format(init_path))


Load init: ../ckpts/mnist_cnn_fl.init


In [10]:
best = 0

for epoch in range(1, args.epochs + 1):
    train(args, model, fog_graph, workers, X_trains, y_trains,
          device, epoch)
    acc = test(args, model, device, test_loader, best, epoch)

    if args.save_model and acc > best:
        best = acc
        torch.save(model.state_dict(), best_path)
        print('Model best  @ {}, acc {}: {}\n'.format(epoch, acc, best_path))
    
if (args.save_model):
    torch.save(model.state_dict(), stop_path)
    print('Model stop: {}'.format(stop_path))


Model best  @ 1, acc 0.1793: ../ckpts/mnist_cnn_fl.best



Model best  @ 2, acc 0.3031: ../ckpts/mnist_cnn_fl.best



Model best  @ 3, acc 0.3945: ../ckpts/mnist_cnn_fl.best



Model best  @ 4, acc 0.4855: ../ckpts/mnist_cnn_fl.best



Model best  @ 5, acc 0.5699: ../ckpts/mnist_cnn_fl.best



...

Model best  @ 195, acc 0.9643: ../ckpts/mnist_cnn_fl.best



Model best  @ 197, acc 0.9652: ../ckpts/mnist_cnn_fl.best



Model best  @ 199, acc 0.9655: ../ckpts/mnist_cnn_fl.best



Train Epoch: 200 	Loss: 0.133071 +- 0.016108



Test set: Average loss: 0.1211, Accuracy: 9634/10000 (96.34%) ==> 96.55%
Model stop: ../ckpts/mnist_cnn_fl.stop
